In [5]:
import os
import json

import torch
from torchvision import transforms
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from torchvision.datasets import ImageFolder
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import DataLoader

class ConfusionMatrix(object):
    def __init__(self, num_classes: int, labels: list):
        self.matrix = np.zeros((num_classes, num_classes))
        self.num_classes = num_classes
        self.labels = labels

    def update(self, preds, labels):
        for p, t in zip(preds, labels):
            self.matrix[p, t] += 1

    def summary(self):
        # calculate accuracy
        sum_TP = 0
        for i in range(self.num_classes):
            sum_TP += self.matrix[i, i]
        acc = sum_TP / np.sum(self.matrix)
        print("the model accuracy is ", acc)

        # precision, recall, specificity
        table = PrettyTable()
        table.field_names = ["", "Precision", "Recall", "Specificity"]
        for i in range(self.num_classes):
            TP = self.matrix[i, i]
            FP = np.sum(self.matrix[i, :]) - TP
            FN = np.sum(self.matrix[:, i]) - TP
            TN = np.sum(self.matrix) - TP - FP - FN
            UA = round(TP / (TP + FN),3) if TP + FN != 0 else 0.
            PA = round(TP / (TP + FP),3) if TP + FP != 0 else 0.
            EA = round(((TP+FP)*(TP+FN)+(TN+FP)*(TN+FN))/(TP+TN+FP+FN)**2,3) if TP+TN+FP+FN != 0 else 0.
            Kappa = (acc-EA)/(1-EA)
            print('User’s Accuracy is:',UA)
            print('Producer’s Accuracy is:',PA)
            print('Expected Accuracy is:',EA)
            print('Kappa Statistic is:',Kappa)
            Precision = round(TP / (TP + FP), 3) if TP + FP != 0 else 0.
            Recall = round(TP / (TP + FN), 3) if TP + FN != 0 else 0.
            Specificity = round(TN / (TN + FP), 3) if TN + FP != 0 else 0.
            table.add_row([self.labels[i], Precision, Recall, Specificity])
        print(table)

    def plot(self):
        matrix = self.matrix
        print(matrix)
        plt.imshow(matrix, cmap=plt.cm.Blues)

        # 设置x轴坐标label
        plt.xticks(range(self.num_classes), self.labels, rotation=45)
        # 设置y轴坐标label
        plt.yticks(range(self.num_classes), self.labels)
        # 显示colorbar
        plt.colorbar()
        plt.xlabel('True Labels')
        plt.ylabel('Predicted Labels')
        plt.title('Confusion matrix')

        # 在图中标注数量/概率信息
        thresh = matrix.max() / 2
        for x in range(self.num_classes):
            for y in range(self.num_classes):
                # 注意这里的matrix[y, x]不是matrix[x, y]
                info = int(matrix[y, x])
                plt.text(x, y, info,
                         verticalalignment='center',
                         horizontalalignment='center',
                         color="white" if info > thresh else "black")
        plt.tight_layout()
        plt.show()


if __name__ == '__main__':
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)

    data_transform = transforms.Compose([transforms.Resize(224),transforms.ToTensor(),transforms.Normalize(mean=[.5,.5,.5],std=[.5,.5,.5])])


    dataset_train=ImageFolder('add_label/label_picture/train',transform=transforms)     #训练数据集
    batch_size = 4
    validate_loader = torch.utils.data.DataLoader(dataset_train,
                                                  batch_size=batch_size, shuffle=True,
                                                  num_workers=0)
    
    
    
    normalize=transforms.Normalize(mean=[5,5,5],std=[5,5,5])  #规范化
    transform=transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor(),normalize]) #数据处理

    dataset_valid=ImageFolder('add_label/label_picture/valid',transform=transform)     #验证或测试数据集

    print(dataset_valid.class_to_idx)
    
    #torch自带的标准数据集加载函数
    dataloader_test=DataLoader(dataset_valid,batch_size=4,shuffle=True,num_workers=0,drop_last=True)
    
    
    net=models.resnet50()#使用迁移学习，加载预训练权重

    in_features=net.fc.in_features
    net.fc=nn.Sequential(nn.Linear(in_features,36),nn.Linear(36,4))#将最后的全连接改为（36，4），对应五个树种
    
    model_weight_path = "best_model.pth"
    assert os.path.exists(model_weight_path), "cannot find {} file".format(model_weight_path)  #判断语句，判断权重是否存在。
    
    net.load_state_dict(torch.load(model_weight_path))
    net.to(device)

    try:
        json_file = open('class_indices.json', 'r')
        class_indict = json.load(json_file)
    except Exception as e:
        print(e)
        exit(-1)


    labels = [label for _, label in class_indict.items()]
    confusion = ConfusionMatrix(num_classes=4, labels=labels)
    net.eval()
    with torch.no_grad():
        for val_data in tqdm(dataloader_test):
            val_images, val_labels = val_data
            outputs = net(val_images.to(device))
            outputs = torch.softmax(outputs, dim=1)
            outputs = torch.argmax(outputs, dim=1)
            confusion.update(outputs.to("cpu").numpy(), val_labels.to("cpu").numpy())
    confusion.plot()
    confusion.summary()

cuda:0
{'fir': 0, 'pine': 1, 'spruce': 2, 'trembling aspen': 3}


RuntimeError: Error(s) in loading state_dict for ResNet:
	size mismatch for fc.1.weight: copying a param with shape torch.Size([5, 36]) from checkpoint, the shape in current model is torch.Size([4, 36]).
	size mismatch for fc.1.bias: copying a param with shape torch.Size([5]) from checkpoint, the shape in current model is torch.Size([4]).

In [2]:
pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 30.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [matplotlib]6 [matplotlib]
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install prettytable

Note: you may need to restart the kernel to use updated packages.


In [6]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 20 13:38:16 2023

@author: Lenovo
"""

import time
from torch.utils.tensorboard import SummaryWriter
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.nn as nn
import torch

print("是否使用GPU训练：{}".format(torch.cuda.is_available()))    #打印是否采用gpu训练
if torch.cuda.is_available:
    print("GPU名称为：{}".format(torch.cuda.get_device_name()))  #打印相应的gpu信息
normalize=transforms.Normalize(mean=[5,5,5], std=[5,5,5]) #规范化
transform=transforms.Compose([transforms.Resize((64, 64)),transforms.ToTensor(),normalize]) #数据处理
dataset_train=ImageFolder('add_label/label_picture/train',transform=transform)     #训练数据集
# print(dataset_tran[0])
dataset_test=ImageFolder('add_label/label_picture/test',transform=transform)     #验证或测试数据集
dataset_valid=ImageFolder('add_label/label_picture/valid',transform=transform)

# print(dataset_train.classer)#返回类别
print(dataset_train.class_to_idx)                               #返回类别及其索引
# print(dataset_train.imgs)#返回图片路径
print(dataset_test.class_to_idx)
print(dataset_valid.class_to_idx)
train_data_size=len(dataset_train)                              #放回数据集长度
test_data_size=len(dataset_test)
valid_data_size=len(dataset_valid)
print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))
print("验证数据集的长度为：{}".format(test_data_size))
#torch自带的标准数据集加载函数
dataloader_train=DataLoader(dataset_train,batch_size=4,shuffle=True,num_workers=0,drop_last=True)
dataloader_test=DataLoader(dataset_test,batch_size=4,shuffle=True,num_workers=0,drop_last=True)
dataloader_valid=DataLoader(dataset_valid,batch_size=4,shuffle=True,num_workers=0,drop_last=True)
#模型加载
model_ft=models.resnet50(pretrained=True)#使用迁移学习，加载预训练权重
# print(model_ft)

in_features=model_ft.fc.in_features
model_ft.fc=nn.Sequential(nn.Linear(in_features,36),nn.Linear(36,4))#将最后的全连接改为（36，6），使输出为六个小数，对应四个树种的置信度


model_ft=model_ft.cuda()#将模型迁移到gpu

#pytorch2.0编译模型部分
# model_ft=torch.compile(model_ft)

#优化器
loss_fn=nn.CrossEntropyLoss()

loss_fn=loss_fn.cuda()  #将loss迁移到gpu
learn_rate=0.0001       #设置学习率
optimizer=torch.optim.SGD(model_ft.parameters(),lr=learn_rate,momentum=0.001)#可调超参数


total_train_step=0
total_test_step=0
total_valid_step=0

epoch=50              #迭代次数
writer=SummaryWriter("logs")
best_acc=-1
ss_time=time.time()

for i in range(epoch):
    start_time = time.time()
    print("--------第{}轮训练开始---------".format(i+1))
    model_ft.train()
    for data in dataloader_train:
        imgs,targets=data
        imgs=imgs.cuda()
        targets=targets.cuda()
        outputs=model_ft(imgs)
        loss=loss_fn(outputs,targets)

        optimizer.zero_grad()   #梯度归零
        loss.backward()         #反向传播计算梯度
        optimizer.step()        #梯度优化

        total_train_step=total_train_step+1
        if total_train_step%100==0:#一轮时间过长可以考虑加一个
            end_time=time.time()
            print("使用GPU训练100次的时间为：{}".format(end_time-start_time))
            print("训练次数：{},loss:{}".format(total_train_step,loss.item()))

    model_ft.eval()
    
    total_test_loss=0
    total_test_accuracy=0
    
    total_train_loss=0
    total_train_accuracy=0

    total_valid_loss = 0
    total_valid_accuracy = 0
    with torch.no_grad():       #验证数据集时禁止反向传播优化权重
      #记录训练集精度和损失值
        for data in dataloader_train:
            imgs,targets=data
            imgs = imgs.cuda()
            targets = targets.cuda()
            outputs=model_ft(imgs)
            train_loss=loss_fn(outputs,targets)
            total_train_loss=total_train_loss+train_loss.item()
            train_accuracy=(outputs.argmax(1)==targets).sum()
            total_train_accuracy=total_train_accuracy+train_accuracy
        print("整体训练集上的loss：{}(越小越好,与上面的loss无关此为测试集的总loss)".format(total_train_loss))
        print("整体训练集上的正确率：{}(越大越好)".format(total_train_accuracy / len(dataset_train)))
       
        for data in dataloader_test:
            imgs,targets=data
            imgs = imgs.cuda()
            targets = targets.cuda()
            outputs=model_ft(imgs)
            test_loss=loss_fn(outputs,targets)
            total_test_loss=total_test_loss+test_loss.item()
            test_accuracy=(outputs.argmax(1)==targets).sum()
            total_test_accuracy=total_test_accuracy+test_accuracy
        print("整体测试集上的loss：{}(越小越好,与上面的loss无关此为测试集的总loss)".format(total_test_loss))
        print("整体测试集上的正确率：{}(越大越好)".format(total_test_accuracy / len(dataset_test)))

        for data in dataloader_valid:
            imgs, targets = data
            imgs = imgs.cuda()
            targets = targets.cuda()
            outputs = model_ft(imgs)
            valid_loss = loss_fn(outputs, targets)
            total_valid_loss = total_valid_loss + valid_loss.item()
            valid_accuracy = (outputs.argmax(1) == targets).sum()
            total_valid_accuracy = total_valid_accuracy + valid_accuracy
        print("整体验证集上的loss：{}(越小越好,与上面的loss无关此为测试集的总loss)".format(total_valid_loss))
        print("整体验证集上的正确率：{}(越大越好)".format(total_valid_accuracy / len(dataset_valid)))

        # 记录到 TensorBoard
        writer.add_scalar("Train Accuracy", total_train_accuracy / len(dataset_train), i)
        writer.add_scalar("Train Loss", total_train_loss, i)
        writer.add_scalar("Test Accuracy", total_test_accuracy/len(dataset_test), i)
        writer.add_scalar("Test Loss", total_test_loss, i)
        writer.add_scalar("Valid Accuracy", total_valid_accuracy / len(dataset_valid), i)
        writer.add_scalar("Valid Loss", total_valid_loss, i)

        total_test_step = total_test_step + 1
        if total_test_accuracy > best_acc:   #保存迭代次数中最好的模型
            print("已修改模型")
            best_acc = total_test_accuracy
            torch.save(model_ft.state_dict(), "best_model.pth")      #只保留权重的参数即可 
# # 绘制训练和测试精度和损失曲线
# plt.figure(figsize=(12, 4))
# plt.subplot(1, 2, 1)
# plt.plot(total_train_loss, label='Train Loss')
# plt.plot(total_test_loss, label='Test Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()


# plt.subplot(1, 2, 2)
# plt.plot(total_train_accuracy / len(dataset_train), label='Train Accuracy')
# plt.plot(total_test_accuracy/len(dataset_test), label='Test Accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy (%)')
# plt.legend()
# plt.show()
ee_time=time.time()
zong_time=ee_time-ss_time
print("训练总共用时:{}h:{}m:{}s".format(int(zong_time//3600),int((zong_time%3600)//60),int(zong_time%60))) #打印训练总耗时
writer.close()

是否使用GPU训练：True
GPU名称为：NVIDIA GeForce RTX 4060 Ti
{'fir': 0, 'pine': 1, 'spruce': 2, 'trembling aspen': 3}
{'fir': 0, 'pine': 1, 'spruce': 2, 'trembling aspen': 3}
{'fir': 0, 'pine': 1, 'spruce': 2, 'trembling aspen': 3}
训练数据集的长度为：597
测试数据集的长度为：85
验证数据集的长度为：85


/venv/main/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/venv/main/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


--------第1轮训练开始---------
使用GPU训练100次的时间为：3.7175514698028564
训练次数：100,loss:1.4598197937011719
整体训练集上的loss：208.05015486478806(越小越好,与上面的loss无关此为测试集的总loss)
整体训练集上的正确率：0.27805694937705994(越大越好)
整体测试集上的loss：28.72661602497101(越小越好,与上面的loss无关此为测试集的总loss)
整体测试集上的正确率：0.38823530077934265(越大越好)
整体验证集上的loss：59.442251563072205(越小越好,与上面的loss无关此为测试集的总loss)
整体验证集上的正确率：0.2705882489681244(越大越好)
已修改模型
--------第2轮训练开始---------
使用GPU训练100次的时间为：1.7766978740692139
训练次数：200,loss:1.2805020809173584
整体训练集上的loss：201.19963294267654(越小越好,与上面的loss无关此为测试集的总loss)
整体训练集上的正确率：0.33835846185684204(越大越好)
整体测试集上的loss：28.40150487422943(越小越好,与上面的loss无关此为测试集的总loss)
整体测试集上的正确率：0.364705890417099(越大越好)
整体验证集上的loss：56.68860191106796(越小越好,与上面的loss无关此为测试集的总loss)
整体验证集上的正确率：0.364705890417099(越大越好)
--------第3轮训练开始---------
使用GPU训练100次的时间为：0.06865406036376953
训练次数：300,loss:1.2972763776779175
使用GPU训练100次的时间为：3.5831215381622314
训练次数：400,loss:1.269704818725586
整体训练集上的loss：197.2267471551895(越小越好,与上面的loss无关此为测试集的总loss)
整体训练集上的正确率：0.37855947